In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.core import XGBoostError
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import plot_importance

import shap

import csv

import cupy as cp

import gc

In [2]:
!python3 --version

Python 3.11.14


In [3]:
xgb.__version__

'3.1.2'

In [4]:
shap.__version__

'0.50.0'

In [5]:
np.__version__

'2.4.2'

In [6]:
df = pd.read_csv("../../story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon. It was bi...,./hidden_states/prompt_1.npz,273
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon. He loved ...,./hidden_states/prompt_1.npz,246
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Sam....,./hidden_states/prompt_1.npz,397
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. He was a ...,./hidden_states/prompt_1.npz,294
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,296
...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,315
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,270
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,206
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,375


In [7]:
hidden_states_by_cl = {}
curr_labels = {}
NUM_PROMPTS = 10
layer = 0

In [8]:
max_story_len = max(df[:NUM_PROMPTS*1000]["len_generated_story"])
max_story_len

522

In [9]:
df[df["len_generated_story"] >= max_story_len]

,prompt_id,prompt,story,hidden_state_file,len_generated_story
9851,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522


In [10]:
min_story_len = min(df["len_generated_story"])
min_story_len

37

In [11]:
context_levels = [0] #get hs used to generate second token
context_levels.extend([i for i in range(23, max_story_len, 25)]) #actual context level is context_level + 2 (gather hs to generate the context_level + 2 token)

In [12]:
hidden_states_by_cl = {f"cl_{cl+2}": [] for cl in context_levels}
curr_labels = {f"cl_{cl+2}": [] for cl in context_levels}

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'../../../llamatales-xgboost-ii/hidden_states/prompt_{prompt_id}.npz') as loaded_data:

        for i in tqdm(range(1000)):
            hs = loaded_data[f"arr_{i}"]  # load once

            for context_level in context_levels:
                if context_level >= hs.shape[0]:
                    continue

                key = f"cl_{context_level + 2}"

                hidden_states_by_cl[key].append(
                    hs[context_level][layer][0].astype("float32")
                )
                curr_labels[key].append(prompt_id - 1)


 44%|██████████████████████████████████▉                                             | 436/1000 [00:09<00:12, 45.02it/s]


KeyboardInterrupt: 

In [ ]:
def safe_split(total, train_ratio=0.8):
    train = int(train_ratio * total)   # floor
    test  = total - train              # leftover
    return train, test

def build_dataset(curr_context_level_hs, curr_labels, train_ratio=0.8):
    """
    Groups samples by prompt_id, splits safely into train/test,
    and returns X_train, y_train, X_test, y_test.
    """

    unique_ids = sorted(set(curr_labels))

    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []

    for pid in unique_ids:

        # get samples for this prompt id
        mask = (curr_labels == pid)
        X_pid = curr_context_level_hs[mask]
        y_pid = curr_labels[mask]

        total = len(X_pid)
        train_n, test_n = safe_split(total, train_ratio)

        # split
        X_train_list.append(X_pid[:train_n])
        y_train_list.append(y_pid[:train_n])

        X_test_list.append(X_pid[train_n:])
        y_test_list.append(y_pid[train_n:])

    # concatenate all prompt-id blocks
    X_train = np.concatenate(X_train_list, axis=0)
    y_train = np.concatenate(y_train_list, axis=0)
    X_test  = np.concatenate(X_test_list, axis=0)
    y_test  = np.concatenate(y_test_list, axis=0)

    # return cp.array(X_train), cp.array(y_train), cp.array(X_test), cp.array(y_test)
    return X_train, y_train, X_test, y_test
    

In [ ]:
np.array(hidden_states_by_cl[f"cl_2"]).shape

In [ ]:
print(np.array(curr_labels[f"cl_2"]).shape)

In [ ]:
with open("results-shap_values.csv", "w+", newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',')
        header = ['Layer', 'Context_Level', 'Most_Important_Test_Feature']
        prompt_headers = []
        for i in range(1, 11):
            prompt_headers.extend([f"Prompt_{i}_Most_Important_Test_Feature"])
    
        header.extend(prompt_headers)
        csv_writer.writerow(header)
for context_level in context_levels[1:]:
    print(f"Optimizing Layer {layer} at Context Level {context_level + 2}")
    cl_hs_array = hidden_states_by_cl[f"cl_{context_level + 2}"]
    curr_label_set = curr_labels[f"cl_{context_level + 2}"]
    # print(cl_hs_array[0].shape)
    # print(cl_hs_array[-1].shape)
    #1x512

    print(np.array(cl_hs_array).shape)
    for hs in range(len(cl_hs_array)):
        cl_hs_array[hs] = cl_hs_array[hs].flatten()

    curr_context_level_hs = np.array(cl_hs_array)
    print(curr_context_level_hs.shape)

    curr_label_set = np.array(curr_label_set)

    unique_ids = sorted(set(curr_label_set))

    if(len(unique_ids) < 10): break
    
    X_train, y_train, X_test, y_test = build_dataset(curr_context_level_hs, curr_label_set)

    X_train_opt, X_valid, y_train_opt, y_valid = train_test_split(X_train, y_train, test_size = 0.2, random_state=42)

    #split into train and test and see how many samples of each class are in the test set (might explain 0.0 acc performance in test set).
    print(pd.Series(y_train).value_counts())
    print(pd.Series(y_test).value_counts())
    print(pd.Series(y_train_opt).value_counts())
    print(pd.Series(y_valid).value_counts())

    print("Train Data Shape: ", X_train.shape)
    print("Test Data Shape: ", X_test.shape)
    print("Optimizer Train Data Shape: ", X_train_opt.shape)

    X_train = cp.array(X_train)
    y_train = cp.array(y_train)
    X_test = cp.array(X_test)
    y_test = cp.array(y_test)

    X_train_opt = cp.array(X_train_opt)
    y_train_opt = cp.array(y_train_opt)
    X_valid = cp.array(X_valid)
    y_valid = cp.array(y_valid)

    #Test 1
    # classifier = XGBClassifier(max_depth = 3, 
    #                            reg_alpha = 10, 
    #                            reg_lambda = 10, 
    #                            gamma = 10, 
    #                            subsample = 0.75,
    #                            colsample_bytree = 0.75,
    #                            eta = 0.01,
    #                            n_estimators = 500,
    #                            # min_child_weight = 20,
    #                            seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = len(unique_ids), tree_method='hist', device='cuda')

    #Test 2
    # classifier = XGBClassifier(max_depth = 6, 
    #                            reg_alpha = 10, 
    #                            reg_lambda = 10, 
    #                            gamma = 10, 
    #                            subsample = 0.75,
    #                            colsample_bytree = 0.75,
    #                            eta = 0.01,
    #                            n_estimators = 500,
    #                            min_child_weight = 20,
    #                            seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = len(unique_ids), tree_method='hist', device='cuda')

    classifier = XGBClassifier(seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = len(unique_ids), tree_method='hist', device='cuda')
    
    
    classifier.fit(X_train, y_train)
    # preds_train = classifier.predict(X_train)
    # preds = classifier.predict(X_test)
    preds_train = np.argmax(classifier.predict_proba(X_train), axis=1)
    preds = np.argmax(classifier.predict_proba(X_test), axis=1)

    plot_importance(classifier, max_num_features=20, importance_type='gain')
    plt.show()

    # plot_importance(classifier, max_num_features=20, importance_type='cover')
    # plt.show()

    # plot_importance(classifier, max_num_features=20, importance_type='weight')
    # plt.show()


    used_features = classifier.get_booster().get_score().keys()
    print("# of Used Features", len(used_features))


    explainer = shap.TreeExplainer(classifier)
    shap_values = explainer(cp.asnumpy(X_test))

    prompt_feats = []
    for i in range(shap_values.values.shape[2]):
        print(f"Class {i}")
        shap.summary_plot(shap_values[:, :, i], cp.asnumpy(X_test))
        shap.plots.bar(shap_values[:, :, i])

        # mean_abs_shap = np.abs(shap_values[:, :, i]).mean(axis=0)
        # most_important_idx = mean_abs_shap.argmax()
        # most_important_feature = X.columns[most_important_idx]

        most_important_prompt_feature = np.argsort(-np.abs(shap_values.values[:, :, i]).mean(axis = 0))[0]

        print(f"Most Important Feature for Prompt {i}: ", most_important_prompt_feature)
        prompt_feats.append(most_important_prompt_feature)

    #Overall
    # mean_abs_shap = np.abs(shap_values).mean(axis=(0, 2))
    # most_important_idx = mean_abs_shap.argmax()
    # most_important_feature = X.columns[most_important_idx]

    most_important_overall_feature = np.argsort(-np.abs(shap_values.values).mean(axis=(0, 2)))[0]

    print(f"Most Important Overall Feature: ", most_important_overall_feature)

    train_accuracy = np.mean(cp.array(preds_train) == y_train)
    accuracy = np.mean(cp.array(preds) == y_test)

    print(f"Train Accuracy: {train_accuracy}")
    print(f"Test Accuracy: {accuracy}")

    # prompt_accs = []

    for i in range(10):
        mask = y_test == i
        prompt_test = y_test[mask]
        prompt_preds = cp.array(preds)[mask]
        prompt_test_acc = np.mean(prompt_preds == prompt_test)

        mask_train = y_train == i
        prompt_train = y_train[mask_train]
        prompt_preds_train = cp.array(preds_train)[mask_train]
        prompt_train_acc = np.mean(prompt_preds_train == prompt_train)

        print(f"Prompt {i + 1} Train Accuracy: {prompt_train_acc}")
        print(f"Prompt {i + 1} Test Accuracy: {prompt_test_acc}")

        # prompt_accs.append(prompt_train_acc)
        # prompt_accs.append(prompt_test_acc)

    with open("results-shap_values.csv", "a+", newline='') as csvfile:
            csv_writer = csv.writer(csvfile, delimiter=',')
            values = [layer, context_level + 2, most_important_overall_feature]
            values.extend(prompt_feats)
            csv_writer.writerow(values)
            


    del classifier, X_train, y_train, X_test, y_test, preds

    # XGBoost cleanup
    try:
        booster = classifier.get_booster()
        del booster
    except:
        pass
    
    # CuPy cleanup
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    
    gc.collect()

In [ ]:
# np.argsort(-np.abs(shap_values.values[:, :, i]).mean(axis = 0))[0]

In [ ]:
# np.argsort(-np.abs(shap_values.values).mean(axis=(0, 2)))[0]

In [ ]:
shap_values

In [ ]:
shap_values.values

In [ ]:
shap_values.base_values

In [ ]:
shap_values.data

In [ ]:
# for i in range(shap_values.values.shape[2]):
#     print(f"Class {i}")
#     shap.summary_plot(shap_values[:, :, i])

In [ ]:
# class_idx = 0  # choose class

# shap.plots.beeswarm(shap_values[:, :, class_idx])

In [ ]:
# shap.summary_plot(
#     shap_values.values[:, :, class_idx],
#     cp.asnumpy(X_test)
# )

In [ ]:
# for i in range(shap_values.values.shape[2]):
#     print(f"Class {i}")
#     shap.plots.beeswarm(shap_values[:, :, i])

In [ ]:
# global_importance = np.mean(np.abs(shap_values), axis=(0,2))